In [ ]:
import import_ipynb

## Use resize when calc mean, std/ num_epoch = 5 / use transform 1/ data set num: 100 / optim: SGD/ batch: 64

Epoch [1/5], Loss: 0.6829988956451416
Epoch [2/5], Loss: 0.660178005695343
Epoch [3/5], Loss: 0.6521103978157043
Epoch [4/5], Loss: 0.6584694385528564
Epoch [5/5], Loss: 0.6707316040992737
checking accuracy on Training set
got 57 / 100 with accracy 56.99999999999999
checking accuracy on Test set
got 8 / 20 with accracy 40.0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

import os

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 64
num_epochs = 5

In [ ]:
# transform 1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    ToTensor()
])


In [ ]:
# transform 2
# transform = transforms.Compose([
#    transforms.Resize((224, 224)),
#    ToTensor(),
# #    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
#     transforms.Normalize(mean, std)
# ])


In [ ]:
# transform 3
# transform = transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize(mean, std, inplace=False)
# ])

In [ ]:

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)

In [ ]:

print("Checking dataset...")


for i in range(5):
    image, label = dataset[i]
    print(f"Image {i} shape: {image.shape}, Label: {label}")

print("\nChecking train loader...")


for images, labels in train_loader:
    print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
    break  

print("\nChecking test loader...")


for images, labels in test_loader:
    print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
    break  


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Linear(5000, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits


In [ ]:
# model

model = NeuralNetwork().to(device)
print(model)

In [ ]:
# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

In [ ]:

# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
#optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

    # training mode
    model.train()

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)

## Use resize when calc mean, std/ num_epoch = 10 / use transform 1/ data set num: 100 / optim: SGD/ batch: 64

Epoch [1/10], Loss: 0.688021719455719
Epoch [2/10], Loss: 0.6919488906860352
Epoch [3/10], Loss: 0.6964294910430908
Epoch [4/10], Loss: 0.6756751537322998
Epoch [5/10], Loss: 0.6902875304222107
Epoch [6/10], Loss: 0.6616805195808411
Epoch [7/10], Loss: 0.6635900735855103
Epoch [8/10], Loss: 0.6734824776649475
Epoch [9/10], Loss: 0.6482532620429993
Epoch [10/10], Loss: 0.644137978553772
show more (open the raw output data in a text editor) ...

Epoch [10/10], Loss: 0.644137978553772
checking accuracy on Training set
got 62 / 100 with accracy 62.0
checking accuracy on Test set
got 10 / 20 with accracy 50.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 64
num_epochs = 10

# transform 1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    ToTensor()
])


# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Linear(5000, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
#optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

    # training mode
    model.train()

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)

## Use resize when calc mean, std/ num_epoch = 10 / use transform 1/ data set num: 25000 / optim: SGD/ batch: 64

Epoch [1/10], Loss: 0.6884214878082275
Epoch [2/10], Loss: 0.6389276385307312
Epoch [3/10], Loss: 0.6553725600242615
Epoch [4/10], Loss: 0.6433557868003845
Epoch [5/10], Loss: 0.6392791271209717
Epoch [6/10], Loss: 0.659751296043396
Epoch [7/10], Loss: 0.6316360235214233
Epoch [8/10], Loss: 0.600490391254425
Epoch [9/10], Loss: 0.6885417699813843
Epoch [10/10], Loss: 0.6437572240829468
checking accuracy on Training set
got 13286 / 20000 with accracy 66.43
checking accuracy on Test set
got 3225 / 5000 with accracy 64.5

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 64
num_epochs = 10

# transform 1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    ToTensor()
])


# Load data

dataset = CatAndDog('./cat_dog.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 5000]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Linear(5000, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
#optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model).


















## Use randomresizedcrop when calc mean, std/ num_epoch = 10 / use transform 3/ data set num: 100 / optim: SGD/ batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer

Epoch [1/10], Loss: 0.6595055460929871
Epoch [2/10], Loss: 0.581857442855835
Epoch [3/10], Loss: 0.6393834948539734
Epoch [4/10], Loss: 0.8630320429801941
Epoch [5/10], Loss: 0.6885052919387817
Epoch [6/10], Loss: 0.5494183897972107
Epoch [7/10], Loss: 0.5923810601234436
Epoch [8/10], Loss: 0.6690357327461243
Epoch [9/10], Loss: 0.7105026245117188
Epoch [10/10], Loss: 0.6995693445205688
checking accuracy on Training set
got 54 / 100 with accracy 54.0
checking accuracy on Test set
got 9 / 20 with accracy 45.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
# from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# transform 3
transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.4875, 0.4498, 0.4112], [0.2132, 0.2091, 0.2080], inplace=False),
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use randomresizedcrop when calc mean, std/ num_epoch = 10 / use transform 2/ data set num: 100 / optim: SGD/ batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer

Epoch [1/10], Loss: 0.7946253418922424
Epoch [2/10], Loss: 0.6217297911643982
Epoch [3/10], Loss: 0.6169763803482056
Epoch [4/10], Loss: 0.5657680034637451
Epoch [5/10], Loss: 0.514208972454071
Epoch [6/10], Loss: 0.6392225623130798
Epoch [7/10], Loss: 0.41998255252838135
Epoch [8/10], Loss: 0.5450870990753174
Epoch [9/10], Loss: 0.41531461477279663
Epoch [10/10], Loss: 0.37717658281326294
checking accuracy on Training set
got 97 / 100 with accracy 97.0
checking accuracy on Test set
got 10 / 20 with accracy 50.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 10 / use transform 2/ data set num: 100 / optim: SGD/ batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer

Epoch [1/10], Loss: 0.7497851848602295
Epoch [2/10], Loss: 0.6210096478462219
Epoch [3/10], Loss: 0.5349128246307373
Epoch [4/10], Loss: 0.5577841401100159
Epoch [5/10], Loss: 0.7867729067802429
Epoch [6/10], Loss: 0.8586550354957581
Epoch [7/10], Loss: 0.44590994715690613
Epoch [8/10], Loss: 0.4820903241634369
Epoch [9/10], Loss: 0.3990113139152527
Epoch [10/10], Loss: 0.27303779125213623
checking accuracy on Training set
got 99 / 100 with accracy 99.0
checking accuracy on Test set
got 9 / 20 with accracy 45.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 10 / use transform 2/ data set num: 100 / optim: SGD/ batch: 32 / add sigmoid/ dropout location: before input layer, between linear, before output layer

Epoch [1/10], Loss: 0.6924512982368469
Epoch [2/10], Loss: 0.6731902360916138
Epoch [3/10], Loss: 0.7045927047729492
Epoch [4/10], Loss: 0.6608633399009705
Epoch [5/10], Loss: 0.7139399647712708
Epoch [6/10], Loss: 0.6770305633544922
Epoch [7/10], Loss: 0.6662049293518066
Epoch [8/10], Loss: 0.7075626850128174
Epoch [9/10], Loss: 0.6222336292266846
Epoch [10/10], Loss: 0.6222627758979797
checking accuracy on Training set
got 76 / 100 with accracy 76.0
checking accuracy on Test set
got 7 / 20 with accracy 35.0

In [1]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



importing Jupyter notebook from cat_and_dog_dataset.ipynb
importing Jupyter notebook from img_normal.ipynb
Mean: tensor([0.4883, 0.4551, 0.4170])
Standard Deviation: tensor([0.2256, 0.2211, 0.2213])
Epoch [1/10], Loss: 0.6924512982368469
Epoch [2/10], Loss: 0.6731902360916138
Epoch [3/10], Loss: 0.7045927047729492
Epoch [4/10], Loss: 0.6608633399009705
Epoch [5/10], Loss: 0.7139399647712708
Epoch [6/10], Loss: 0.6770305633544922
Epoch [7/10], Loss: 0.6662049293518066
Epoch [8/10], Loss: 0.7075626850128174
Epoch [9/10], Loss: 0.6222336292266846
Epoch [10/10], Loss: 0.6222627758979797
checking accuracy on Training set
got 76 / 100 with accracy 76.0
checking accuracy on Test set
got 7 / 20 with accracy 35.0


## Use resize when calc mean, std/ num_epoch = 10 / use transform 2/ data set num: 25000 / optim: SGD/ batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer/ labels.float() -> labels when crossEntropy

Epoch [1/10], Loss: 0.7786927819252014 
Epoch [2/10], Loss: 0.6254715919494629 
Epoch [3/10], Loss: 0.5060574412345886 
Epoch [4/10], Loss: 0.5186963677406311 
Epoch [5/10], Loss: 0.7118619680404663 
Epoch [6/10], Loss: 0.4992092549800873 
Epoch [7/10], Loss: 0.5119767785072327 
Epoch [8/10], Loss: 0.6405879855155945 
Epoch [9/10], Loss: 0.49774616956710815 
Epoch [10/10], Loss: 0.5938100218772888 
checking accuracy on Training set got 16223 / 20000 with accracy 81.11500000000001 checking accuracy on Test set got 3410 / 5000 with accracy 68.2

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 5000]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels)

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)

